# gradient test
in this notebook we get to know a test witch help us to findout our implementation of 
backpropagation is correct 

## simple function 
lets strat with a simple 2d function J = thetha * x 

In [1]:
import numpy as np 
from testCases import * 

In [2]:
def forward_propagation(x,theta):
    return np.dot(theta,x)

In [3]:
x, theta = 2, 4
J = forward_propagation(x, theta)
print ("J = " + str(J))

J = 8


In [4]:
def backward_propagation(x,theta):
    return x 

In [5]:
dtheta = backward_propagation(x, theta)
print ("dtheta = " + str(dtheta))

dtheta = 2


In [6]:
def gradient_check(x,theta,epsilon=1e-7):
    J_plus  = forward_propagation(x,theta+epsilon)
    J_minus = forward_propagation(x,theta-epsilon)
    gradapprox = (J_plus - J_minus)/(2*epsilon)
    grad  = backward_propagation(x,theta)
    diffrence = np.divide((np.linalg.norm(grad - gradapprox)),(np.linalg.norm(grad) + np.linalg.norm(gradapprox)))
    if diffrence < epsilon:
        print("gradient is correct")
    else:
        print("gradient is wrong !")
    return diffrence

In [7]:
x, theta = 2, 4
difference = gradient_check(x, theta)
print("difference = " + str(difference))

gradient is correct
difference = 2.919335883291695e-10


## n dimension function 

In [8]:
from gc_utils import relu , sigmoid

In [59]:
def forward_propagation_n(X,y,param):
    """
    implementation of n dimension function like network
    return:
    cost -- loss over whole training set 
    cache -- all the information of net in each layer in order of Z , A , W , b
    *** as you can see this dosnt return any y_hat
    """
    # initializing 
    W1 = param["W1"]
    b1 = param["b1"]
    W2 = param["W2"]
    b2 = param["b2"]
    W3 = param["W3"]
    b3 = param["b3"]
    # forward pass
    Z1 = np.dot(W1,X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2,A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3,A2) + b3
    A3 = sigmoid(Z3)
    # compute cost
    try:
        m = y.shape[1]
    except:
        m = len(y)
        
    
    
    log_prob = -(y*np.log(A3))-((1-y)*np.log((1-A3)))
    cost = np.sum(log_prob) / m

    cache_ = (Z1 , A1 , W1 ,b1 , Z2 ,A2 , W2 , b2 ,Z3 , A3 ,W3 , b3)
    return cost , cache_ 

In [61]:
def backward_propagation_n(X,y,cache):
    """
    implementation of n dimension function like net 

    arguments:
    X -- feature matrix 
    y -- ground true label
    cache -- cache from forward pass
    
    return:
    grad -- dictionary of gradiant
    """
    
    # initialization 
    try:
        m = y.shape[1]
    except:
        m = len(y)
    (Z1 , A1 , W1 ,b1 , Z2 ,A2 , W2 , b2 ,Z3 , A3 ,W3 , b3) = cache
    
    # backward pass 
    dZ3 = A3 - y
    dW3 = np.dot(dZ3,A2.T)/m 
    db3 = np.sum(dZ3,axis=1,keepdims=True)/m
    dA2 = np.dot(W3.T,dZ3)
    dZ2 = np.multiply(dA2,np.int64(A2>0))
    dW2 = np.dot(dZ2,A1.T)/m 
    db2 = np.sum(dZ2,axis=1,keepdims=True)/m 
    dA1 = np.dot(W2.T,dZ2)
    dZ1 = np.multiply(dA1,np.int64(A1>0))
    dW1 = np.dot(dZ1,X.T) /m 
    db1 = np.sum(dZ1,axis=1,keepdims=True)/m 

    grads = {
        "dW3":dW3 , "db3":db3 , "dZ3":dZ3 , "dA2":dA2 ,
        "dW2":dW2 , "db2":db2 , "dZ2":dZ2 , "dA1":dA2 ,
        "dW1":dW1 , "db1":db1 , "dZ1":dZ1 
    }

    return grads 

In [13]:
from gc_utils import vector_to_dictionary , dictionary_to_vector , gradients_to_vector

In [48]:
def gradient_check_n(param,grad,X,y,epsilon=1e-7):
    """
    implement gradient check for network with matrix and vector weights

    arguments:
    param -- network parameters (during training or after training)
    grad -- our created gradiants 
    X -- feature matrix 
    y -- ground true labels
    epsilon -- lenth of derivative 

    returns:
    diffrence -- a value witch show how our gradient is ok 

    """
    # create theta 
    theta  , _ = dictionary_to_vector(param)
    theta_plus = np.copy(theta) #+ epsilon
    theta_minus = np.copy(theta) #- epsilon 
    # craete gradapprox
    gradapprox = np.zeros(theta.shape)
    for i in range(theta.shape[0]):
        # theta_plus = np.copy(theta) #+ epsilon
        # theta_minus = np.copy(theta) #- epsilon
        theta_plus[i][0] = theta[i][0] + epsilon
        theta_minus[i][0] = theta[i][0] - epsilon

        J_plus , cache_plus = forward_propagation_n(X,y,vector_to_dictionary(theta_plus))
        J_minus , cache_minus = forward_propagation_n(X,y,vector_to_dictionary(theta_minus))
        gradapprox[i][0] = (J_plus - J_minus) / (2*epsilon)

        theta_plus[i][0] = theta[i][0] 
        theta_minus[i][0] = theta[i][0] 
    
    grad_vectore = gradients_to_vector(grad)
    numerator = np.linalg.norm(grad_vectore - gradapprox)
    denumerator = np.linalg.norm(grad_vectore) + np.linalg.norm(gradapprox)
    difference = numerator / denumerator

    if difference <= 2*epsilon:
        print("gradiants is ok")
        print(difference)
    else:
        print("gradiants is wrong!")
        print(difference)

    return difference

    

In [62]:
X, Y, parameters = gradient_check_n_test_case()

cost, cache = forward_propagation_n(X, Y, parameters)
gradients = backward_propagation_n(X, Y, cache)
difference = gradient_check_n(parameters, gradients, X, Y)

gradiants is ok
1.1909939370580298e-07
